# Web Scraping "Página 12"
## Primer paso: request
### **Importar Librerías**
Lo primero que vamos a realizar es importar las librerías, en este caso vamos a necesitar requests para conectar la url y bs para realizar el scrapping. Aparte importamos pandas para crear y manipular un dataset con la información obtenida. Importamos además matplotlib para realizar lagunas gráficas.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### **Establecer conexión con la url**
Lo siguiente que vamos a realizar una solicitud a la url del portal del diario página12, para ello utilizamos de la libreria requests que importamos en las primeras líneas el método 
```python 
.get()
```

In [3]:
url = 'https://www.pagina12.com.ar/'
p12 = requests.get(url)

Para chequear que la solicitud fue establecida con éxito, ejecutamos:
    
```python
.status_code

```
Si está todo ok nos devería volver 200, para más info buscar sobre códigos de status HTTP

In [4]:
p12.status_code


200

Ahora que ya establecimos la conexión con la URL, vamos a imprimir el contenido html de la página con la comando:
```python
.text
```

In [5]:
print(p12.text)

<!DOCTYPE html><html class="no-js " lang="es"><head><meta charset="utf-8"><title>Página | 12: La otra mirada sobre Argentina y el mundo</title><meta name="google-site-verification" content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg"><link rel="canonical" href="https://www.pagina12.com.ar"><script type="application/ld+json">{"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}</script><meta property="description" name="description" content="Información sobre Argentina y el mundo. Noticias en fotos y videos de los principales hechos y acontecimientos del país. Análisis, opinión y entrevistas."><meta property="fb:pages" n

## Segundo paso: BeautifulSoup
### Limpiar el texto html de la página
Lo siguiente que tenemos que hacer es extraer la información del código html que nos parezca relevante. Para ello vamos a utilizar la librería BeautifulSoup.\
Con esta librería lo que vamos a realizar es trozar nuestro código html y quedarnos con las partes que contengan la información que querramos.
Lo primero que hay que hacer es crear un objeto soup, lo hacemos con el método:
```python
BeautifulSoup(web_scrapear.text, 'lmxl')
 ```

In [6]:
# Lo guardamos en una variable
sopa = BeautifulSoup(p12.text, 'lxml')

In [7]:
# Chequeamos el tipo del dato
type(sopa)

bs4.BeautifulSoup

In [8]:
# Imprimos el contenido que guardamos en la variable y aplicamos método .prettify() para verlo más ordenado
print(sopa.prettify())

<!DOCTYPE html>
<html class="no-js" lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   Página | 12: La otra mirada sobre Argentina y el mundo
  </title>
  <meta content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg" name="google-site-verification"/>
  <link href="https://www.pagina12.com.ar" rel="canonical"/>
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}
  </script>
  <meta content="Información sobre Argentina y el mundo. Noticias en fotos y videos de los principales hechos y acontecimientos del país. Análisis, opinión y entrevistas." name="description" property="des

### Seleccionamos y extraemos la información que queremos screapear
Lo siguiente que vamos a realizar es seleccionar la información que queremos escrapear, en este caso deseamos selecionar todas las secciones que nos ofrece el portal para realizar un scraping de todos los articulos que contienen cada una de ellas.\
Para extraer la información lo que vamos a hacer es utilizar el método .find(), con el cual vamos a poder definir que tipo de etiqueta o tag queremos extraer. Luego sobre el objeto que crea el método .find(), lo que podemos hacer es usar el método .find_all ('tag').

```python
.find('div', attrs={'class' : 'aca la caracteristica de ese class'}).find_all('tag')
```

In [9]:
# Extraemos las secciones y las guardamos en una variable
secciones = sopa.find('ul', attrs={'class' : 'horizontal-list main-sections hide-on-dropdown'}).find_all('li')

In [10]:
# Guardamos el primer elemento para poder trabajarlo más facilmente y luego replicarlo a las demás secciones
seccion = secciones[0]
# Visualizamos el contenido
print (seccion)

<li class="p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/el-pais">El país</a></li>


Analizamos lo que nos devuelve y observamos que nos interesan dos cosas, por un lado el título de la sección, en este caso "El país", y por otro lado la URL, la cual nos va a permitir a nosotros poder automatizar el programa de scraping.\
Para extraer esa información vamos a utilizar el método .get_text() para extraer la información del título de la sección y el método .get('href') para extraer la dirección de la url
```python
.get_text()
.get('href')
```

In [11]:
# Extraemos el nombre de la sección.
seccion.a.get_text()

'El país'

In [12]:
# Extraemos la URL
seccion.a.get('href')

'https://www.pagina12.com.ar/secciones/el-pais'

### **Creamos una lista con todas las secciones**
Ahora que ya extrajimos la información de una sección, podemos replicar lo mismo para las demás secciones.\
Para ello vamos a crear una lista que contenga cada una de ellas, accedemos a cada una de ellas y nos quedamos con el atributo ('href) y lo replicamos con un ciclo for a las demás listas que guardamos en la variables secciones.

In [13]:
# Creamos la lista, 
links_secciones = [seccion.a.get('href') for seccion in secciones]

In [14]:
# Imprimimos y chequeamos que ya haya salido todo ok
links_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/ciencia',
 'https://www.pagina12.com.ar/secciones/el-mundo']

### **Extraemos las notas de cada seccion**
Ahora que ya tenemos todas las url que necesitamos de cada una de las secciones, procedemos a extraer las notas que contienen cada una de ellas. Para ello volvemos a quedarnos con un elemento para luego replicarlo a los demás.


In [15]:
# Guardamos al primer elemento en una variable, con un nuevo request, luego pasamos el mismo método a toda la lista de links
sec = requests.get(links_secciones[0])

In [16]:
# Chequeamos que este todo ok con la conexión, debe devolver 200
sec.status_code

200

### Extraer texto de las notas
Ahora lo que vamos a realizar es extraer las url a las notas.\
Debemos repetir el proceso que hicimos anteriormente, por lo cual creamos una sopa de la sección, y luego analizamos el contenido que queremos extraer. \


In [17]:
# Lo que tenemos que hacer ahora es volver a partir el elemento con el BeautifulSoup, de tla manera creamos un objeto soup de la sección
sopa_seccion = BeautifulSoup(sec.text, 'lxml')

In [18]:
# Imprimimos para ver que tenemos
print (sopa_seccion.prettify())

<!DOCTYPE html>
<html amp="" lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   El país | Página12
  </title>
  <!-- DUST PATH: /usr/src/app/src/widgets/fc_jsonLD.dust/ -->
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}
  </script>
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "NewsArticle","mainEntityOfPage": {"@type": "WebPage","@id": "https://www.pagina12.com.ar/secciones/el-pais"},"headline": "El país | Página12","isAccessibleForFree": "False","image": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/lo

* Si observamos la web de página12 podemos obserrvar que tenemos una noticia principal que es la que aparece primero y en mayor dimensión. Vemos que se encuentra dentro de ```<div>```y corresponde a una clase  ```<article-item_content>```.\
* Vamos a extraer esa información del artículo promocionado y la vamos a guardar en una variable.

In [19]:
# Extraemos la información del artículo promocionado y lo guardamos en una variable con el método find y find_all,
art_top = sopa_seccion.find('div', attrs={'class' : "article-item__content-footer-wrapper gutter-small deco-bar-here-left is-mobile-top"})

In [20]:
print (art_top)

<div class="article-item__content-footer-wrapper gutter-small deco-bar-here-left is-mobile-top"><div class="article-item__content"><!-- Title --><h1 class="title-list"><a href="/495387-acabar-con-la-interna">Acabar con la interna</a></h1><!-- Kicker --></div><div class="article-item__footer"><!-- Author --><div class="author hide-on-mobile"><span>Por </span><a href="https://www.pagina12.com.ar/autores/659-eduardo-aliverti">Eduardo Aliverti</a></div><!-- Date --><div class="date hide-on-mobile">06 de noviembre de 2022</div><!-- Author Mobile --><div class="author is-display-inline p12-separator-right-gray hide-on-desktop"><span>Por </span><a href="https://www.pagina12.com.ar/autores/659-eduardo-aliverti">Eduardo Aliverti</a></div><!-- Date Mobile --><div class="date is-display-inline hide-on-desktop">06 de noviembre de 2022</div></div></div>


* Advertimos que también existen dos artículos más que podríamos llamarlos márginales, que también están patrocinados, por lo cual, vamos a guaradrlos en una variable.

In [21]:
sopa_seccion.find('div', attrs={'class' : "articles-list is-grid-col2 grid-mobile-row"})

<div class="articles-list is-grid-col2 grid-mobile-row"><!-- DUST PATH: /usr/src/app/src/templates/partials/amp/lists/articles_list_item.dust/ --><!-- DUST PATH: /usr/src/app/src/templates/partials/amp/articles/featured_article.dust/ --><article class="article-item article-item--featured"><!-- Image --><div class="article-item__header deco-bar-here-bottom is-mobile-left"><a href="/495359-garantizar-que-la-derecha-no-vuelva-a-la-argentina-es-un-deb"><!-- DUST PATH: /usr/src/app/node_modules/frontend-core/views/widgets/fc_displayImg_amp.dust/ --><amp-img alt=" (Fuente: Adrián Pérez)" class="" height="313" layout="responsive" src="https://images.pagina12.com.ar/styles/focal_3_2_470x313/public/2022-11/666408-fbde130f-6fb6-4829-ba4c-fa7236f37d85_0.jpg?itok=jw2XxE3l" srcset="https://images.pagina12.com.ar/styles/focal_3_2_470x313/public/2022-11/666408-fbde130f-6fb6-4829-ba4c-fa7236f37d85_0.jpg?itok=jw2XxE3l 470w, https://images.pagina12.com.ar/styles/focal_3_2_300x200/public/2022-11/666408-f

In [22]:
art_marginales = sopa_seccion.find('div', attrs={'class' : "articles-list is-grid-col2 grid-mobile-row"})

In [23]:
art_marginales

<div class="articles-list is-grid-col2 grid-mobile-row"><!-- DUST PATH: /usr/src/app/src/templates/partials/amp/lists/articles_list_item.dust/ --><!-- DUST PATH: /usr/src/app/src/templates/partials/amp/articles/featured_article.dust/ --><article class="article-item article-item--featured"><!-- Image --><div class="article-item__header deco-bar-here-bottom is-mobile-left"><a href="/495359-garantizar-que-la-derecha-no-vuelva-a-la-argentina-es-un-deb"><!-- DUST PATH: /usr/src/app/node_modules/frontend-core/views/widgets/fc_displayImg_amp.dust/ --><amp-img alt=" (Fuente: Adrián Pérez)" class="" height="313" layout="responsive" src="https://images.pagina12.com.ar/styles/focal_3_2_470x313/public/2022-11/666408-fbde130f-6fb6-4829-ba4c-fa7236f37d85_0.jpg?itok=jw2XxE3l" srcset="https://images.pagina12.com.ar/styles/focal_3_2_470x313/public/2022-11/666408-fbde130f-6fb6-4829-ba4c-fa7236f37d85_0.jpg?itok=jw2XxE3l 470w, https://images.pagina12.com.ar/styles/focal_3_2_300x200/public/2022-11/666408-f

* Ahora lo que vamos a hacer es extraer el contenido de todas las demás listas de noticias que aparecen más abajo, y nuevamente vamos a guardarlo en una variable.

In [24]:
art_lista = sopa_seccion.find('article', attrs={'class' : "article-item article-item--teaser"})

In [25]:
print(art_lista.a.get('href'))

/495290-dejen-de-jugar-a-los-tristes-y-ofendidos


### Crear función para extraer todo el contenido que nos interese de cada nota
Ahora vamos a crear una función que nos devuelva el contenido de nos interesa de cada noticia, en este caso la url para luego automatizar el scrap.

In [92]:
def extraer_notas (sopa):
    '''
    Función que relice un objeto de BeautifulSoup de una página de secciones
    y devuelve una lista de URLs a las notas de esa sección
    '''
    lista_notas = []
    
    # Obtengo el artículo promocionado
    art_promocionado = sopa.find ('div', attrs={'class' : "article-item__content-footer-wrapper gutter-small deco-bar-here-left is-mobile-top"})
    for promocionado in art_promocionado.find_all ('a'):
        lista_notas.append(promocionado.get('href'))
    
    # Obtengo los artículos marginales
    articulos_marginales = sopa.find ('div', attrs={'class' : "articles-list is-grid-col2 grid-mobile-row"})
    for link in articulos_marginales.find_all('a'):
        lista_notas.append(link.get('href'))
        
    # Obtengo las url de la lista de noticias
    lista_noticias = sopa.find('article', attrs={'class' : "article-item article-item--teaser"})
    for noticia in lista_noticias.find_all('a'):
        lista_notas.append(noticia.get('href'))
        
    return lista_notas

Al analizar la lista que nos devuelve la función advertimos que falta la dirección httml y tenemos muchos links repetidos, esto es devido a que no solamente tenemos la dirección url en el título sino también en el resumen y en algunos casos también en la imagen. Asimismo, tenemos algunos links a los perfiles de los y las periodistas que redactaron las notas.\
Nos va a tocar limpiar el contenido de la lista.\
Ahora que tenemos la nueva lista limpia, y sin los links https, vamos a agregar a cada elemento de la lista la dirección de la web de página12.\
Lo hacemos con un ciclo for que itere por cada elemento de la nueva lista, y guardamos el resultado en otra nueva lista. Luego creamos un función para automatizar el proceso y ya estamos listos para continuar con el scrap.

In [28]:
# Para limpiar la lista que nos devuelve la función vamos a usar pandas, para facilitarlo.
lista_notas = extraer_notas(sopa_seccion)
#len(lista_notas)
lista_notas = list(set(lista_notas))

#type(lista_notas)
lista_nueva_notas = []
lista_links_notas = []
for elemento in lista_notas:
    elemento = str(elemento)
    if elemento.find('http') and elemento.find('None'):
        lista_nueva_notas.append(elemento)
        for nuevo_elemento in lista_nueva_notas:
            elemento = str(elemento)
        lista_links_notas.append ('https://www.pagina12.com.ar' + elemento)
        
lista_links_notas

['https://www.pagina12.com.ar/495355-cordoba-en-un-confuso-episodio-hirieron-de-un-balazo-al-inte',
 'https://www.pagina12.com.ar/secciones/el-pais?page=1',
 'https://www.pagina12.com.ar/495290-dejen-de-jugar-a-los-tristes-y-ofendidos',
 'https://www.pagina12.com.ar/495213-maximo-kirchner-en-el-congreso-del-pj',
 'https://www.pagina12.com.ar/495270-argentina-sera-sede-de-la-tercera-edicion-del-foro-mundial-d',
 'https://www.pagina12.com.ar/495359-garantizar-que-la-derecha-no-vuelva-a-la-argentina-es-un-deb',
 'https://www.pagina12.com.ar/495269-atentado-a-cfk-el-poder-judicial-mira-para-otro-lado',
 'https://www.pagina12.com.ar/495387-acabar-con-la-interna',
 'https://www.pagina12.com.ar/495179-larreta-critico-a-bullrich-sin-mencionarla',
 'https://www.pagina12.com.ar/495236-lula-la-historia-comun-durante-20-anos',
 'https://www.pagina12.com.ar/495221-mauricio-macri-quedo-en-el-centro-de-las-investigaciones-por',
 'https://www.pagina12.com.ar/495267-ley-de-humedales-diputados-del-frent

### Corrección y manejo de errores:
Ahora lo que debemos realizar es continuar con el scraping y generar una función que realice la misma acción previa en cada una de las secciones, con el problema que nos encontramos, fue que al realizarla, y al haber algunos links caídos, se nos cortaba la ejecución del ciclo for. Al encontrarme con este error, lo que advertí fue la manejar y procesar los errores.\
Para manejar los errores podemos utilizar la librería request, y crear una condicional, habíamos dicho que si estaba todo ok, nos devolvía el código 200, por tal manera podemos crear una conición if que nos devuelva el booleano de comparación.

In [29]:
r = requests.get(lista_links_notas[0])

if r.status_code == 200:
    #procesamos la respuesta
    print ('procesamos')
else:
    print ('informamos el error...')
    #informar el error
try:
    requests.get(url_caida)
except Exception as e:
    print ('Error al hacer requests')
    print (e)
    print ('\n')

procesamos
Error al hacer requests
name 'url_caida' is not defined




In [30]:
try:
    art_top.a.get('href')
except:
    pass

In [31]:
r = requests.get(links_secciones[0])
if r.status_code == 200:
    # Procesamos la respuesta
    print('procesamos..')
else:
     # Informar el error
    print('informamos...')

procesamos..


### Extracción contenido de cada nota
Ahora que ya tenemos los links a cada una de las notas de cada sección guardados en una lista, vamos a proceder a acceder al contenido de cada nota y extraer todo lo que querramos.\
Para ello lo primero que vamos a hacer es quedarnos con la primer url y guardarlo en una variable, para trabajar más tranquilos.

In [32]:
url_nota = lista_links_notas[0]
url_nota

'https://www.pagina12.com.ar/495355-cordoba-en-un-confuso-episodio-hirieron-de-un-balazo-al-inte'

In [54]:
try:
    nota = requests.get(url_nota)
    if nota.status_code == 200:
        sopa_nota = BeautifulSoup(nota.text, 'lxml')
        # Extraemos el título
        titulo = sopa_nota.find('div', attrs={'class' : 'col 2-col'}).find('h1')
        volanta = sopa_nota.find('div', attrs= {'class' : 'col 2-col'}).find('h4')
        copete = sopa_nota.find('div', attrs= {'class' : 'col 2-col'}).find('h3')
        fecha = sopa_nota.find('div', attrs= {'class' : 'date modification-date'}).find('time')
        autor = sopa_nota.find('div', attrs= {'class' : 'authors-inner-list'}).find('a')
        cuerpo = sopa_nota.find('div', attrs = {'class' : 'article-main-content article-text'})
        #imagen = 
        print('El título es: ', titulo.text)
        print ('La volanda es: ', volanta.text)
        print ('El copete dice: ', copete.text)
        print ('Fue escrita el día: ', fecha.text)
        print ('Por el autor: ', autor.text)
        print (cuerpo.text)
except Exception as e:
    print ('Error: ')
    print (e)
    print('\n')

El título es:  Córdoba: en un confuso episodio, hirieron de un balazo al intendente de San Francisco 
La volanda es:  Fue operado y perdió un riñón
El copete dice:  El ataque ocurrió a las 2.30 de esta madrugada, cuando el funcionario salía de una fiesta por el Día del Bancario. Está internado en terapia intensiva y su estado es grave. Quedó detenido un odontólogo, esposo de una empleada del municipio. Los investigadores creen que el móvil del ataque es una cuestión personal.
Fue escrita el día:  6 de noviembre de 2022 - 14:15
Por el autor:  Gimena Bugallo
El intendente de la localidad cordobesa de San Francisco, Damián Bernarte, fue herido de un balazo en el abdomen esta madrugada cuando salía de una fiesta desarrollada en el estadio Superdomo local y por el ataque quedó formalmente detenido un odontólogo, esposo de una empleada municipal, informaron fuentes policiales.El hecho ocurrió pasadas las 2.30 cuando el jefe comunal dejaba el predio donde se desarrolló un evento de festejo de

Ahora que ya extrajimos la información que queremos de toda la nota, vamos a crear una función para que lo replique a las demás notas y a todas las demás secciones.\
Luego hacemos una función unificando todo y tendríamos listo el proyecto.

In [102]:
def extraer_contenido_nota (sopa):
    
    # Creamos un diccionario para guardar toda la información
    diccionario_notas = {}
    
    # Extraemos la fecha
    fecha = sopa.find('div', attrs= {'class' : 'date modification-date'})
    if fecha:
        diccionario_notas['fecha'] = fecha.text
    else:
        diccionario_notas['fecha'] = None
    
    # Extraemos la volanta
    volanta = sopa.find('div', attrs= {'class' : 'section-2-col article-header'})
    if volanta:
        for volanta in volanta.find_all('h4'):
            diccionario_notas['volanta'] = volanta.text
    else:
        diccionario_notas['volanta'] = None
    
    # Extraemos el título
    titulo = sopa.find('div', attrs={'class' : 'section-2-col article-header'})
    if titulo:
        for titulo in titulo.find_all ('h1'):
            diccionario_notas['titulo'] = titulo.text
    else:
        diccionario_notas['titulo'] = None
    
    # Extraemos el copete
    copete = sopa.find('div', attrs= {'class' : 'section-2-col article-header'})
    if copete:
        for copete in copete.find_all('h3'):
            diccionario_notas['copete'] = copete.text
    else:
        diccionario_notas['copete'] = None
    
    # Extraemos el cuerpo
    cuerpo = sopa.find('div', attrs = {'class' : 'article-main-content article-text'})
    if cuerpo:
        diccionario_notas['cuerpo'] = cuerpo.text
    else:
        diccionario_notas['cuerpo'] = None
    
    # Extraemos el autor
    autor = sopa.find('div', attrs= {'class' : 'authors-inner-list'})
    if autor:
        diccionario_notas['autor'] = autor.text
    else:
        diccionario_notas['autor'] = None
        
    return diccionario_notas

In [104]:
extraer_contenido_nota (sopa_nota)

{'fecha': '6 de noviembre de 2022 - 14:15',
 'volanta': 'Fue operado y perdió un riñón',
 'titulo': 'Córdoba: en un confuso episodio, hirieron de un balazo al intendente de San Francisco\xa0',
 'copete': 'El ataque ocurrió a las 2.30 de esta madrugada, cuando el funcionario salía de una fiesta por el Día del Bancario. Está internado en terapia intensiva y su estado es grave. Quedó detenido un odontólogo, esposo de una empleada del municipio. Los investigadores creen que\xa0el móvil del ataque es una cuestión personal.',
 'cuerpo': 'El intendente de la localidad cordobesa de San Francisco, Damián Bernarte, fue herido de un balazo en el abdomen esta madrugada cuando salía de una fiesta desarrollada en el estadio Superdomo local y por el ataque quedó formalmente detenido un odontólogo, esposo de una empleada municipal, informaron fuentes policiales.El hecho ocurrió pasadas las 2.30 cuando el jefe comunal dejaba el predio donde se desarrolló un evento de festejo del Día del Bancario, y se 

In [89]:
def scraping_nota (url):
    try:
        nota = requests.get (url)
    except Exception as e:
        print ('Error en el scraping por la URL', url)
        print (e)
        print ('\n')
        return None
    
    if nota.status_code != 200:
        print(f'Error obteniendo la nota {url}')
        print (f'status Code: {nota.status_code}')
        return None
    
    sopa_nota = BeautifulSoup (nota.text, 'lxml')
    
    diccionario_csv = extraer_contenido_nota (sopa_nota)
    diccionario_csv['url'] = url
    
    return diccionario_csv

In [90]:
scraping_nota (url_nota)

{'fecha': '6 de noviembre de 2022 - 14:15',
 'volanta': 'Fue operado y perdió un riñón',
 'titulo': 'Córdoba: en un confuso episodio, hirieron de un balazo al intendente de San Francisco\xa0',
 'copete': 'El ataque ocurrió a las 2.30 de esta madrugada, cuando el funcionario salía de una fiesta por el Día del Bancario. Está internado en terapia intensiva y su estado es grave. Quedó detenido un odontólogo, esposo de una empleada del municipio. Los investigadores creen que\xa0el móvil del ataque es una cuestión personal.',
 'cuerpo': 'El intendente de la localidad cordobesa de San Francisco, Damián Bernarte, fue herido de un balazo en el abdomen esta madrugada cuando salía de una fiesta desarrollada en el estadio Superdomo local y por el ataque quedó formalmente detenido un odontólogo, esposo de una empleada municipal, informaron fuentes policiales.El hecho ocurrió pasadas las 2.30 cuando el jefe comunal dejaba el predio donde se desarrolló un evento de festejo del Día del Bancario, y se 

In [50]:
links_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/ciencia',
 'https://www.pagina12.com.ar/secciones/el-mundo']

In [93]:
notas = []
notas_limpias = []
notas_limpias_link = []

for link in links_secciones:
    try:
        r = requests.get (link)
        if r.status_code == 200:
            sopa = BeautifulSoup (r.text, 'lxml')
            notas.extend(extraer_notas(sopa))
            for elemento in notas:
                elemento = str(elemento)
                if elemento.find('http') and elemento.find('None'):
                    notas_limpias.append(elemento)
                    for nuevo_elemento in notas_limpias:
                        elemento = str(elemento)
                        notas_limpias_link.append ('https://www.pagina12.com.ar' + elemento)
                        notas_limpias_link = list(set(notas_limpias_link))
            
        else:
            print ('No se pudo obtener la sección', link)
   
    except Exception as e:
        print ('Error: ')
        print (e)
        print('\n')

notas_limpias_link

Error: 
'NoneType' object has no attribute 'find_all'




['https://www.pagina12.com.ar/494996-daniel-scioli-esta-eleccion-abre-una-oportunidad-unica-en-la',
 'https://www.pagina12.com.ar/495273-compensacion-fiscal',
 'https://www.pagina12.com.ar/495355-cordoba-en-un-confuso-episodio-hirieron-de-un-balazo-al-inte',
 'https://www.pagina12.com.ar/495351-clima-en-buenos-aires-el-pronostico-del-tiempo-para-este-dom',
 'https://www.pagina12.com.ar/495257-liga-espanola-barcelona-despidio-a-pique-con-una-victoria',
 'https://www.pagina12.com.ar/495282-creo-que-el-papa-podra-visitarnos-nuevamente',
 'https://www.pagina12.com.ar/495272-las-elecciones-mas-inclusivas-en-la-historia-de-estados-unid',
 'https://www.pagina12.com.ar/495255-el-acuerdo-xl-los-rebeldes-y-las-carpetas',
 'https://www.pagina12.com.ar/485032-jupiter-alcanzara-su-punto-mas-cercano-a-la-tierra-en-casi-6',
 'https://www.pagina12.com.ar/495068-nueva-inversion-china-para-producir-litio',
 'https://www.pagina12.com.ar/secciones/deportes?page=1',
 'https://www.pagina12.com.ar/495099-se-

In [52]:
len(notas_limpias_link)

72

In [733]:
len (notas)


# Para limpiar la lista que nos devuelve la función vamos a usar pandas, para facilitarlo.
#lista_notas = extraer_notas(sopa_seccion)
#len(lista_notas)
#lista_notas = list(set(lista_notas))

#type(lista_notas)
lista_nueva_notas_todas = []
lista_links_notas_todas = []
for elemento in notas:
    elemento = str(elemento)
    if elemento.find('http') and elemento.find('None'):
        lista_nueva_notas_todas.append(elemento)
        for nuevo_elemento in lista_nueva_notas_todas:
            elemento = str(elemento)
        lista_links_notas_todas.append ('https://www.pagina12.com.ar' + elemento)

lista_links_notas_todas = list(set(lista_links_notas_todas))
lista_links_notas_todas

['https://www.pagina12.com.ar/495248-una-nueva-edicion-de-la-cumbre-climatica-entre-el-escepticis',
 'https://www.pagina12.com.ar/495027-el-papa-francisco-renovo-su-pedido-por-la-paz-mundial',
 'https://www.pagina12.com.ar/483815-cientificos-captaron-las-imagenes-mas-nitidas-de-los-anillos',
 'https://www.pagina12.com.ar/495108-aguanta-la-paz-cambiaria',
 'https://www.pagina12.com.ar/495262-arte-popular-por-los-derechos-humanos',
 'https://www.pagina12.com.ar/495272-las-elecciones-mas-inclusivas-en-la-historia-de-estados-unid',
 'https://www.pagina12.com.ar/495387-acabar-con-la-interna',
 'https://www.pagina12.com.ar/495247-premier-league-julian-alvarez-sigue-haciendo-goles-en-el-cit',
 'https://www.pagina12.com.ar/495259-liga-italiana-napoli-no-para-de-ganar-y-sigue-puntero',
 'https://www.pagina12.com.ar/495221-mauricio-macri-quedo-en-el-centro-de-las-investigaciones-por',
 'https://www.pagina12.com.ar/495269-atentado-a-cfk-el-poder-judicial-mira-para-otro-lado',
 'https://www.pagina

In [105]:
data = []

for i, nota in enumerate (notas_limpias_link):
    print(f'Estamos escrapendo la nota {i}/{len(notas_limpias_link)}')
    data.append(scraping_nota(nota))

Estamos escrapendo la nota 0/72
Estamos escrapendo la nota 1/72
Estamos escrapendo la nota 2/72
Estamos escrapendo la nota 3/72
Estamos escrapendo la nota 4/72
Estamos escrapendo la nota 5/72
Estamos escrapendo la nota 6/72
Estamos escrapendo la nota 7/72
Estamos escrapendo la nota 8/72
Estamos escrapendo la nota 9/72
Estamos escrapendo la nota 10/72
Estamos escrapendo la nota 11/72
Estamos escrapendo la nota 12/72
Estamos escrapendo la nota 13/72
Estamos escrapendo la nota 14/72
Estamos escrapendo la nota 15/72
Estamos escrapendo la nota 16/72
Estamos escrapendo la nota 17/72
Estamos escrapendo la nota 18/72
Estamos escrapendo la nota 19/72
Estamos escrapendo la nota 20/72
Estamos escrapendo la nota 21/72
Estamos escrapendo la nota 22/72
Estamos escrapendo la nota 23/72
Estamos escrapendo la nota 24/72
Estamos escrapendo la nota 25/72
Estamos escrapendo la nota 26/72
Estamos escrapendo la nota 27/72
Estamos escrapendo la nota 28/72
Estamos escrapendo la nota 29/72
Estamos escrapendo l

In [71]:
len(data)

72

In [106]:
df = pd.DataFrame(data)


In [107]:
df.head(25)

fecha  \
0     5 de noviembre de 2022 - 01:31   
1     6 de noviembre de 2022 - 00:29   
2     6 de noviembre de 2022 - 14:15   
3     6 de noviembre de 2022 - 09:36   
4     6 de noviembre de 2022 - 00:29   
5     6 de noviembre de 2022 - 00:29   
6     6 de noviembre de 2022 - 00:29   
7     6 de noviembre de 2022 - 03:57   
8   26 de septiembre de 2022 - 15:18   
9     5 de noviembre de 2022 - 01:31   
10                              None   
11    5 de noviembre de 2022 - 01:31   
12                              None   
13    7 de noviembre de 2022 - 00:07   
14    5 de noviembre de 2022 - 01:31   
15    6 de noviembre de 2022 - 00:29   
16    5 de noviembre de 2022 - 01:31   
17      30 de agosto de 2022 - 15:08   
18  26 de septiembre de 2022 - 14:01   
19    5 de noviembre de 2022 - 02:41   
20    6 de noviembre de 2022 - 04:05   
21    6 de noviembre de 2022 - 04:20   
22    5 de noviembre de 2022 - 01:31   
23    6 de noviembre de 2022 - 17:52   
24    6 de noviembre de 2022 - 00:29   

                                              volanta  \
0   Encuentro del embajador argentino en Brasil co...   
1   Producción afectada por heladas en zona cordil...   
2                       Fue operado y perdió un riñón   
3                     Servicio Meteorológico Nacional   
4   El defensor se retiró con el triunfo 2-0 sobre...   
5   Brasil se abre al mundo después del triunfo el...   
6   Según Human Rights Campaign la comunidad de vo...   
7   El sendero de "freezado y descongelamiento pro...   
8   El planeta resaltará como un punto especialmen...   
9           2200 millones de dólares en dos proyectos   
10                                               None   
11  Bolsonaro emitió un video ordenando levantar l...   
12                                               None   
13  Los hermanos Pieres se impusieron con claridad...   
14  Al menos cuatro palestinos muertos por fuerzas...   
15  Pegadas a los marcos de las majas de Goya, dos...   
16  Para el diplomático su país vive un "extraordi...   
17          Lo que sucede con el cuerpo de un donante   
18  Animales que protagonizaron hitos y avances ci...   
19                  Tres meses con el dólar planchado   
20                Cerró el congreso del PJ bonaerense   
21           Las 4 claves del piquete en Comodoro Py    
22  Massa anunció la construcción de una línea de ...   
23                         Duras críticas al magnate    
24  María Rachid, secretaria de la Federación Arge...   

                                               titulo  \
0   Daniel Scioli: "Esta elección abre una oportun...   
1                                 Compensación fiscal   
2   Córdoba: en un confuso episodio, hirieron de u...   
3   Clima en Buenos Aires: el pronóstico del tiemp...   
4   Liga española: Barcelona despidió a Piqué con ...   
5      "Creo que el Papa podrá visitarnos nuevamente"   
6   Las elecciones mas inclusivas en la historia d...   
7   La estrategia de Massa para los precios: quién...   
8   Júpiter alcanzará su punto más cercano a la Ti...   
9           Nueva inversión china para producir litio   
10                                               None   
11        Se terminan los bloqueos de rutas en Brasil   
12                                               None   
13  Abierto Argentino de polo: Ellerstina debutó c...   
14               Escalada bélica en la Franja de Gaza   
15  Activistas ecologistas protestaron en el Museo...   
16  Carlos Faría, canciller de Venezuela: "Nuestro...   
17  Estudiar la vida a través de la muerte: ¿Le do...   
18  De Laika a Balto, los perros que hicieron hist...   
19                         ¿Aguanta la paz cambiaria?   
20  Máximo Kirchner: “Dejen de jugar a los tristes...   
21  La trama detrás del atentado a CFK que el Pode...   
22                    "Se avanza con orden y energía"   
23  La ONU advierte a Elon Musk que Twitter debe "...   
24  "La Marcha es una fiesta desde la cual exigimo...   

                        

In [780]:
df.to_csv('scraping_notas_pagina12.csv')